# Data Cleaning and Setup

This notebook marks the beginning of my meteorological forecasting project during my internship at **MeteoGalicia**. My first task was to create a clean and well-structured dataset based on the raw data files provided by my tutors. This step was essential to ensure that the dataset was ready for model training and further analysis.

All data processing and model training were performed remotely on the **CESGA** (*Centro Europeo de Supercomputación de Galicia*) high-performance computing cluster.

To access CESGA, I first established a secure **VPN** connection using the snx command-line utility, which allowed CESGA to identify and authorize my device. Once connected, I used **SSH** to access the cluster and standard Bash commands to navigate directories, manage files, and execute Python scripts.

Below is the basic process I used to connect to CESGA via SSH after the VPN was successfully established.

and 

![Alt text](../../../Imagenes/dropout.png)